<a href="https://colab.research.google.com/github/Marcin19721205/WSBNeuronowe/blob/main/SiecLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stworzenie i wizualizacja obiektu

In [16]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# ============================================================
# PARAMETRY SIMULACJI / OBIEKTU
# ============================================================
N           = 5000     # liczba kroków czasowych
dt          = 0.1      # krok czasowy [s]
t           = np.arange(N) * dt

K_proc      = 1.0       # wzmocnienie obiektu
zeta        = 0.4       # współczynnik tłumienia (0<zeta<1 → tłumienie)
omega_n     = 0.15      # częstość naturalna [rad/s] (~1/T)
T_dom       = 1.0 / omega_n   # umowna "stała czasowa" dla opóźnienia

delay_ratio = 0.8       # ułamek stałej czasowej
delay_steps = max(1, int(delay_ratio * T_dom / dt))

print(f"T_dom ≈ {T_dom:.3f} s")
print(f"Delay steps = {delay_steps} (≈ {delay_ratio*100:.1f}% T_dom)")

# ============================================================
# RNG + SZUM: persystentny, skośny, z outlierami
# ============================================================
rng = np.random.default_rng(42)

def generate_persistent_skewed_noise(N,
                                     phi=0.9,
                                     base_sigma=0.3,
                                     skew_factor=0.6,
                                     p_out=0.01,
                                     out_scale=4.0):
    eps  = rng.normal(0.0, base_sigma, size=N)
    noise = np.zeros(N)
    for k in range(1, N):
        noise[k] = phi * noise[k-1] + eps[k]

    skew_part = np.exp(skew_factor * noise)
    skew_part = skew_part - np.mean(skew_part)
    noise_skewed = noise + 0.7 * skew_part

    mask_out = rng.random(N) < p_out
    jumps    = rng.normal(0, out_scale * base_sigma, size=N)
    noise_skewed[mask_out] += jumps[mask_out]

    return noise_skewed

noise = generate_persistent_skewed_noise(N)

# ============================================================
# SYGNAŁ STERUJĄCY MV:
#  - start z MV = 0
#  - skok 0 -> 50% i trzymanie przez ~6*T_dom (faza STEP)
#  - potem PRBS skalowany do T_dom (faza PRBS)
# ============================================================
MV = np.zeros(N)

# czas ustalania: 6 * T_dom (w próbkach)
settle_steps = int(6 * T_dom / dt)
settle_steps = min(settle_steps, N)

# pierwszy skok 0 -> 50%
if settle_steps > 1:
    MV[0] = 0.0
    MV[1:settle_steps] = 50.0
else:
    MV[:] = 50.0

# parametry PRBS w krotnościach T_dom
prbs_min_T_factor = 0.5
prbs_max_T_factor = 2.0

prbs_min_len = max(1, int(prbs_min_T_factor * T_dom / dt))
prbs_max_len = max(prbs_min_len + 1, int(prbs_max_T_factor * T_dom / dt))

prbs_low     = 10.0
prbs_high    = 90.0

current_level = prbs_high
k = settle_steps

while k < N:
    seg_len = int(rng.integers(prbs_min_len, prbs_max_len + 1))
    end_idx = min(N, k + seg_len)
    MV[k:end_idx] = current_level
    current_level = prbs_low if current_level == prbs_high else prbs_high
    k = end_idx

print(f"PRBS segment length in samples: [{prbs_min_len}, {prbs_max_len}]")
print(f"PRBS segment length in seconds: [{prbs_min_len*dt:.2f}, {prbs_max_len*dt:.2f}]")

# ============================================================
# MODEL 2 RZĘDU Z TŁUMIENIEM + OPÓŹNIENIE
# ============================================================
x1 = np.zeros(N)  # CV
x2 = np.zeros(N)  # pochodna

for k in range(1, N):
    if k >= delay_steps:
        u_delayed = MV[k - delay_steps]
    else:
        u_delayed = MV[0]

    dx1 = x2[k-1]
    dx2 = -2.0 * zeta * omega_n * x2[k-1] - (omega_n**2) * x1[k-1] \
          + K_proc * (omega_n**2) * u_delayed

    x1[k] = x1[k-1] + dt * dx1
    x2[k] = x2[k-1] + dt * dx2

CV_clean = x1
CV = CV_clean + noise

# ============================================================
# FAZA TESTU: STEP / PRBS
# ============================================================
phase = np.empty(N, dtype=object)
phase[:settle_steps] = "STEP"
phase[settle_steps:] = "PRBS"

# można też mieć wersję numeryczną, jeśli chcesz pod ML:
# phase_id = np.zeros(N, dtype=int)
# phase_id[settle_steps:] = 1

# ============================================================
# DATAFRAME
# ============================================================
df = pd.DataFrame({
    "t":        t,
    "MV":       MV,
    "CV":       CV,
    "CV_clean": CV_clean,
    "noise":    noise,
    "phase":    phase
    # "phase_id": phase_id,   # jeśli jednak wolisz int
})

print(df.head())
print(df["phase"].value_counts())

# ============================================================
# WIZUALIZACJA
# ============================================================
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=df["t"], y=df["MV"],
                          mode="lines", name="MV (sterowanie)"))
fig1.add_trace(go.Scatter(x=df["t"], y=df["CV"],
                          mode="lines", name="CV (z szumem)"))
fig1.add_trace(go.Scatter(x=df["t"], y=df["CV_clean"],
                          mode="lines", name="CV_clean", line=dict(dash="dot")))

# pionowa linia w miejscu przejścia STEP -> PRBS
fig1.add_vline(x=settle_steps * dt,
               line=dict(dash="dash"),
               annotation_text="START PRBS",
               annotation_position="top right")

fig1.update_layout(
    title="Obiekt 2 rzędu z opóźnieniem + szum (STEP + PRBS, z fazą)",
    xaxis_title="t [s]",
    yaxis_title="Wartość",
)
fig1.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df["t"], y=df["noise"],
                          mode="lines", name="noise"))
fig2.update_layout(
    title="Szum (persystentny, skośny, z outlierami)",
    xaxis_title="t [s]",
    yaxis_title="noise"
)
fig2.show()


T_dom ≈ 6.667 s
Delay steps = 53 (≈ 80.0% T_dom)
PRBS segment length in samples: [33, 133]
PRBS segment length in seconds: [3.30, 13.30]
     t    MV        CV  CV_clean     noise phase
0  0.0   0.0 -0.037219       0.0 -0.037219  STEP
1  0.1  50.0 -0.468718       0.0 -0.468718  STEP
2  0.2  50.0 -0.115870       0.0 -0.115870  STEP
3  0.3  50.0  0.299440       0.0  0.299440  STEP
4  0.4  50.0 -0.555184       0.0 -0.555184  STEP
phase
PRBS    4600
STEP     400
Name: count, dtype: int64


Identyfikacja obiektu - dekonowulcja w dziedzinie częstotliwości

In [18]:
# ============================================================
# IDENTYFIKACJA DEAD TIME METODĄ DEKONWOLUCJI (FFT)
# u(t) = MV, y(t) = CV_clean
# ============================================================

# 1) bierzemy tylko fragment PRBS (lepsza stacjonarność)
mask_prbs = (df["phase"] == "PRBS")
u = df.loc[mask_prbs, "MV"].values.astype(float)       # wejście
y = df.loc[mask_prbs, "CV_clean"].values.astype(float) # wyjście "bez szumu"

# opcjonalnie: odjęcie średniej (żeby DC nie dominowało w FFT)
u = u - u.mean()
y = y - y.mean()

N_id = len(u)
print(f"Liczba próbek do identyfikacji: {N_id}")

# 2) zero-padding (żeby uniknąć zawijania splotu)
n_fft = 2**int(np.ceil(np.log2(2 * N_id)))   # najbliższa potęga 2 > 2*N
u_pad = np.zeros(n_fft)
y_pad = np.zeros(n_fft)
u_pad[:N_id] = u
y_pad[:N_id] = y

# 3) transformaty Fouriera
U = np.fft.fft(u_pad)
Y = np.fft.fft(y_pad)

# 4) dekonwolucja w dziedzinie częstotliwości:
#    H(ω) = Y(ω) / U(ω) – z lekką regularizacją, żeby nie dzielić przez ~0
eps = 1e-6 * np.max(np.abs(U))
H = Y * np.conj(U) / (np.abs(U)**2 + eps)

# 5) odpowiedź impulsowa: h[n] = IFFT{H(ω)}
h = np.fft.ifft(H).real  # interesuje nas część rzeczywista

# 6) kroimy sensowny zakres odpowiedzi impulsowej
#    (np. pierwsze 400 próbek ≈ 40 s przy dt = 0.1)
n_imp = min(400, len(h))
t_imp = np.arange(n_imp) * dt
h_trunc = h[:n_imp]

# 7) odpowiedź skokowa = całka (suma) z odpowiedzi impulsowej
step_resp = np.cumsum(h_trunc) * dt

# 8) estymacja dead time:
#    zakładamy, że odpowiedź na skok = 0 do chwili L,
#    potem startuje. Bierzemy np. próg 3,5% wartości ustalonej.
g_final = step_resp[-1]             # przybliżona wartość ustalona dla skoku 1
threshold = 0.035 * g_final          # 3,5% poziomu końcowego

idx_dead = np.argmax(step_resp > threshold)
t_dead_est = idx_dead * dt

true_dead = delay_steps * dt

print(f"Estymowany dead time z FFT:  t_dead_est ≈ {t_dead_est:.3f} s")
print(f"Prawdziwy dead time w modelu: t_dead_true ≈ {true_dead:.3f} s")
print(f"Indeks dead time: {idx_dead}, (kroki), vs. {delay_steps} z modelu")

# ============================================================
# WIZUALIZACJA: odpowiedź impulsowa i skokowa z FFT
# ============================================================

fig_imp = go.Figure()
fig_imp.add_trace(go.Scatter(x=t_imp, y=h_trunc,
                             mode="lines", name="h[n] (impulse response)"))
fig_imp.add_vline(x=true_dead,
                  line=dict(dash="dash"),
                  annotation_text="true dead time",
                  annotation_position="top right")
fig_imp.add_vline(x=t_dead_est,
                  line=dict(dash="dot"),
                  annotation_text="est dead time",
                  annotation_position="bottom right")
fig_imp.update_layout(
    title="Odpowiedź impulsowa (z dekonwolucji FFT)",
    xaxis_title="t [s]",
    yaxis_title="h(t)"
)
fig_imp.show()

fig_step = go.Figure()
fig_step.add_trace(go.Scatter(x=t_imp, y=step_resp,
                              mode="lines", name="step response (z FFT)"))
fig_step.add_hline(y=g_final, line=dict(dash="dash"),
                   annotation_text="wartość ustalona (≈ K_proc)",
                   annotation_position="top right")
fig_step.add_hline(y=threshold, line=dict(dash="dot"),
                   annotation_text="próg 2%",
                   annotation_position="bottom right")
fig_step.add_vline(x=true_dead,
                  line=dict(dash="dash"),
                  annotation_text="true dead time",
                  annotation_position="top left")
fig_step.add_vline(x=t_dead_est,
                  line=dict(dash="dot"),
                  annotation_text="est dead time",
                  annotation_position="bottom left")
fig_step.update_layout(
    title="Odpowiedź skokowa (całka z h[n]) + estymacja dead time",
    xaxis_title="t [s]",
    yaxis_title="y_step(t)"
)
fig_step.show()


Liczba próbek do identyfikacji: 4600
Estymowany dead time z FFT:  t_dead_est ≈ 5.800 s
Prawdziwy dead time w modelu: t_dead_true ≈ 5.300 s
Indeks dead time: 58, (kroki), vs. 53 z modelu
